In [13]:
from frictionless import Resource, Layout, transform, steps
from frictionless.plugins.sql import SqlDialect
from crawl import settings

In [3]:
db_string = f"{settings.DATABASE['drivername']}://{settings.DATABASE['username']}:{settings.DATABASE['password']}@{settings.DATABASE['host']}:{settings.DATABASE['port']}/{settings.DATABASE['database']}"

In [14]:
res = transform(
    Resource(
    path="https://cividi-zurich.eu-central-1.linodeobjects.com/03-mirror/03-swisstopo/pure_adr.csv",
        layout=Layout(
            pick_fields=["BDG_EGID","STN_LABEL","ADR_NUMBER","ZIP_LABEL","ADR_EASTING","ADR_NORTHING"]
        ),
    ),
    steps=[
        steps.field_add(name="ADR_STRING", function=lambda x: f"{x['STN_LABEL']} {x['ADR_NUMBER']}, {x['ZIP_LABEL']}"),
        steps.table_write(
            path=db_string,
            dialect=SqlDialect(
                namespace=settings.DATABASE_PG_SCHEMA,
                table="addresses",
            )
        ),
    ]
)

res.to_petl()

FrictionlessException: [step-error] Step is not valid: "field_add" raises "[source-error] The data source has not supported or has inconsistent contents: I/O operation on closed file."

+-------------+--------------------+------------+------------------------+-------------+--------------+-----------------------------------------------+
| BDG_EGID    | STN_LABEL          | ADR_NUMBER | ZIP_LABEL              | ADR_EASTING | ADR_NORTHING | ADR_STRING                                    |
+=============+====================+============+========================+=============+==============+===============================================+
| '400079541' | 'Adelbodenstrasse' | '227a'     | '3724 Ried (Frutigen)' | '2612581'   | '1155389'    | 'Adelbodenstrasse 227a, 3724 Ried (Frutigen)' |
+-------------+--------------------+------------+------------------------+-------------+--------------+-----------------------------------------------+
| '190685068' | 'Hintergasse'      | '59b'      | '4914 Roggwil BE'      | '2628216'   | '1231863'    | 'Hintergasse 59b, 4914 Roggwil BE'            |
+-------------+--------------------+------------+------------------------+-------------+